In [10]:
import pandas as pd
import numpy as np

maternal_education = pd.read_csv(r"C:\Users\Elias\Final Project\Maternal Data\Cal-ViDa_Birth_01062026 (2).csv")
maternal_age = pd.read_csv(r"C:\Users\Elias\Final Project\Maternal Data\Cal-ViDa_Birth_01062026 (1).csv")
trimester_care = pd.read_csv(r"C:\Users\Elias\Final Project\Maternal Data\Cal-ViDa_Birth_01062026.csv")

maternal_education.shape

(79920, 7)

In [11]:
maternal_age.shape

(133200, 7)

In [12]:
trimester_care.shape

(66600, 7)

In [2]:
maternal_education.head()

,Type_of_Event,Year_of_Birth,Residence_or_Place_of_Birth,ZIP_Code_of_Residence,Education_Level,Total_Births,Last_Data_Refresh
0,Birth,2022,Place of Residence,90001,No High School Diploma,201,12/10/2025
1,Birth,2022,Place of Residence,90001,High School Graduate or GED Completed,259,12/10/2025
2,Birth,2022,Place of Residence,90001,"Some College Credit, No 4-Year Degree",182,12/10/2025
3,Birth,2022,Place of Residence,90001,Bachelor's Degree,47,12/10/2025
4,Birth,2022,Place of Residence,90001,Graduate Degree,<11,12/10/2025


In [3]:
maternal_age.head()

,Type_of_Event,Year_of_Birth,Residence_or_Place_of_Birth,ZIP_Code_of_Residence,Age,Total_Births,Last_Data_Refresh
0,Birth,2022,Place of Residence,90001,15 years and under,<11,12/10/2025
1,Birth,2022,Place of Residence,90001,16 - 17 years,13,12/10/2025
2,Birth,2022,Place of Residence,90001,18 - 19 years,30,12/10/2025
3,Birth,2022,Place of Residence,90001,20 - 24 years,146,12/10/2025
4,Birth,2022,Place of Residence,90001,25 - 29 years,217,12/10/2025


In [4]:
trimester_care.head()

,Type_of_Event,Year_of_Birth,Residence_or_Place_of_Birth,ZIP_Code_of_Residence,Trimester_Prenatal_Care_Began,Total_Births,Last_Data_Refresh
0,Birth,2022,Place of Residence,90001,First Trimester,566,12/10/2025
1,Birth,2022,Place of Residence,90001,Second Trimester,103,12/10/2025
2,Birth,2022,Place of Residence,90001,Third Trimester,29,12/10/2025
3,Birth,2022,Place of Residence,90001,No Prenatal Care,<11,12/10/2025
4,Birth,2022,Place of Residence,90001,Unknown/Not Reported,<11,12/10/2025


In [5]:
# Dropping columns that already exist in birth data
maternal_education = maternal_education.drop(columns=["Type_of_Event", "Year_of_Birth", "Residence_or_Place_of_Birth", "Last_Data_Refresh"])

# Replacing <11 with 5.5, keeping 0 values
maternal_education["Total_Births"] = maternal_education["Total_Births"].replace("<11", "5.5")
maternal_education["Total_Births"] = maternal_education["Total_Births"].replace("0", "0")
maternal_education["Total_Births"] = pd.to_numeric(maternal_education["Total_Births"], errors="coerce")

# Pivot to get education levels as columns
education_pivot = maternal_education.pivot_table(index="ZIP_Code_of_Residence",columns="Education_Level",values="Total_Births",aggfunc="sum",fill_value=0).reset_index()

# Flattening column names
education_pivot.columns = [
    f"Births_{col.replace(' ', '_').replace('/', '_')}" if col != "ZIP_Code_of_Residence" 
    else col 
    for col in education_pivot.columns
]


maternal_age = maternal_age.drop(columns=["Type_of_Event","Year_of_Birth","Residence_or_Place_of_Birth","Last_Data_Refresh"])

# same approach as before
maternal_age["Total_Births"] = maternal_age["Total_Births"].replace("<11", "5.5")
maternal_age["Total_Births"] = maternal_age["Total_Births"].replace("0", "0")
maternal_age["Total_Births"] = pd.to_numeric(maternal_age["Total_Births"], errors="coerce")

age_pivot = maternal_age.pivot_table(index="ZIP_Code_of_Residence",columns="Age",values="Total_Births",aggfunc="sum",fill_value=0).reset_index()

age_pivot.columns = [
    f"Births_{col.replace(' ', '_').replace('-', '_')}" if col != "ZIP_Code_of_Residence"
    else col
    for col in age_pivot.columns
]


trimester_care = trimester_care.drop(columns=["Type_of_Event","Year_of_Birth","Residence_or_Place_of_Birth","Last_Data_Refresh"])


trimester_care["Total_Births"] = trimester_care["Total_Births"].replace("<11", "5.5")
trimester_care["Total_Births"] = trimester_care["Total_Births"].replace("0", "0")
trimester_care["Total_Births"] = pd.to_numeric(trimester_care["Total_Births"], errors="coerce")


trimester_pivot = trimester_care.pivot_table(index="ZIP_Code_of_Residence",columns="Trimester_Prenatal_Care_Began",values="Total_Births",aggfunc="sum",fill_value=0).reset_index()

trimester_pivot.columns = [
    f"Births_{col.replace(' ', '_').replace('/', '_')}" if col != "ZIP_Code_of_Residence"
    else col
    for col in trimester_pivot.columns
]

# Merging with the birth data that we cleaned
birth_data = pd.read_csv(r"C:\Users\Elias\Final Project\Cleaned output data files\cleaned_birth_data_aggregated.csv")
birth_data = birth_data[['ZIP_Code_of_Residence', 'Total_Births_2018_2022', 'Total_LBW_Count_2018_2022', 'LBW_Rate']]
final_merged = birth_data.merge(education_pivot, on="ZIP_Code_of_Residence", how="left"
).merge(age_pivot,on="ZIP_Code_of_Residence",how="left"
).merge(trimester_pivot,on="ZIP_Code_of_Residence",how="left")

# Filling NaN with 0
final_merged = final_merged.fillna(0)

print(f"Final merged shape: {final_merged.shape}")
print(f"\nColumns: {final_merged.columns.tolist()}")
# Overwrite the aggregated birth data file)
final_merged.to_csv(r"C:\Users\Elias\Final Project\Cleaned output data files\cleaned_birth_data_aggregated.csv", index=False)

Final merged shape: (267, 25)

Columns: ['ZIP_Code_of_Residence', 'Total_Births_2018_2022', 'Total_LBW_Count_2018_2022', 'LBW_Rate', "Births_Bachelor's_Degree", 'Births_Graduate_Degree', 'Births_High_School_Graduate_or_GED_Completed', 'Births_No_High_School_Diploma', 'Births_Some_College_Credit,_No_4-Year_Degree', 'Births_Unknown', 'Births_15_years_and_under', 'Births_16___17_years', 'Births_18___19_years', 'Births_20___24_years', 'Births_25___29_years', 'Births_30___34_years', 'Births_35___39_years', 'Births_40___44_years', 'Births_45___49_years', 'Births_50_years_and_over/Unknown', 'Births_First_Trimester', 'Births_No_Prenatal_Care', 'Births_Second_Trimester', 'Births_Third_Trimester', 'Births_Unknown_Not_Reported']


In [6]:
final_merged.head()

,ZIP_Code_of_Residence,Total_Births_2018_2022,Total_LBW_Count_2018_2022,LBW_Rate,Births_Bachelor's_Degree,Births_Graduate_Degree,Births_High_School_Graduate_or_GED_Completed,Births_No_High_School_Diploma,"Births_Some_College_Credit,_No_4-Year_Degree",Births_Unknown,...,Births_30___34_years,Births_35___39_years,Births_40___44_years,Births_45___49_years,Births_50_years_and_over/Unknown,Births_First_Trimester,Births_No_Prenatal_Care,Births_Second_Trimester,Births_Third_Trimester,Births_Unknown_Not_Reported
0,90001,3936.0,308.0,7.825203,197.0,34.0,1494.0,1238.0,902.0,73.0,...,948.0,536.0,139.0,27.5,0.0,3171.0,52.5,552.0,138.0,27.5
1,90002,3839.5,324.5,8.451621,203.0,35.0,1517.0,1099.0,902.0,81.5,...,883.0,481.0,143.0,22.0,5.5,3118.0,33.0,533.0,115.0,27.5
2,90003,5583.5,439.5,7.871407,231.0,62.5,1993.0,1976.0,1216.0,108.0,...,1205.0,723.0,201.0,27.5,0.0,4418.0,79.0,822.0,206.0,59.0
3,90004,2642.0,211.0,7.986374,585.0,329.0,609.0,525.0,507.0,59.5,...,791.0,618.0,215.0,27.5,16.5,2078.0,27.5,307.0,180.0,27.5
4,90005,1537.0,140.0,9.108653,278.0,135.0,336.0,398.0,320.0,35.0,...,442.0,320.0,102.0,22.0,5.5,1235.0,27.5,200.0,27.5,27.5
